In [ ]:
from utils import *
from MySolution_13 import MyClassifier, MyClustering, MyLabelSelection

%load_ext autoreload
%autoreload 2

In [ ]:
syn_data = prepare_synthetic_data()
mnist_data = prepare_mnist_data()

print("Synthetic data shape: ", syn_data["trainX"].shape, syn_data["trainY"].shape)
print("MNIST data shape: ", mnist_data["trainX"].shape, mnist_data["trainY"].shape)

# Task 1 - Classification

In [ ]:
xmin, xmax = np.min(syn_data["trainX"][:, 0]), np.max(syn_data["trainX"][:, 0])
ymin, ymax = np.min(syn_data["trainX"][:, 1]), np.max(syn_data["trainX"][:, 1])

x_linspace = np.linspace(xmin, xmax, 100)
y_linspace = np.linspace(ymin, ymax, 100)
x, y = np.meshgrid(x_linspace, y_linspace)
cartesian = np.stack((x.ravel(), y.ravel()), axis=1)
syn_class = MyClassifier(3)
syn_class.train(syn_data["trainX"], syn_data["trainY"])

plt.scatter(cartesian[:, 0], cartesian[:, 1], c=syn_class.predict(cartesian), s=1)
print(syn_class.predict(cartesian))
plt.scatter(syn_data["trainX"][:, 0], syn_data["trainX"][:, 1], c=syn_data["trainY"])
# plt.scatter(syn_data["testX"][:, 0], syn_data["testX"][:, 1], c=syn_data["testY"])

# Task 2 - Clustering

In [ ]:
syn_clustering = MyClustering(3)
syn_clustering.train(syn_data["trainX"])
plt.scatter(syn_data["trainX"][:, 0], syn_data["trainX"][:, 1], c=syn_clustering.labels)

In [ ]:
syn_classifier = MyClassifier(3)
syn_classifier.train(syn_data["trainX"], syn_data["trainY"])
mnist_classifier = MyClassifier(3)
mnist_classifier.train(mnist_data["trainX"], mnist_data["trainY"])
result1 = {
    "synthetic_test_accuracy": syn_classifier.evaluate(syn_data["testX"], syn_data["testY"]),
    "mnist_test_accuracy": mnist_classifier.evaluate(mnist_data["testX"], mnist_data["testY"])
}

In [ ]:
result2 = {
    "synthetic": {
        "K": [3, 5, 10],
        "clustering_nmi": [],
        "classification_accuracy": [],
    },
    "mnist": {
        "K": [3, 10, 32],
        "clustering_nmi": [],
        "classification_accuracy": []
    },
}

for k1, k2 in zip(result2["synthetic"]["K"], result2["mnist"]["K"]):
    print(f"training synthetic dataset with K = {k1}")
    syn_clustering = MyClustering(k1)
    syn_clustering.train(syn_data["trainX"])
    syn_nmi = syn_clustering.evaluate_clustering(syn_data["trainY"].astype(np.int32))
    syn_acc = syn_clustering.evaluate_classification(
        syn_data["trainY"].astype(np.int32),
        syn_data["testX"].astype(np.int32),
        syn_data["testY"].astype(np.int32),
    )
    result2["synthetic"]["clustering_nmi"].append(syn_nmi)
    result2["synthetic"]["classification_accuracy"].append(syn_acc)

    print(f"training mnist dataset with K = {k2}")
    mnist_clustering = MyClustering(k2)
    mnist_clustering.train(mnist_data["trainX"])
    mnist_nmi = mnist_clustering.evaluate_clustering(
        mnist_data["trainY"].astype(np.int32)
    )
    mnist_acc = mnist_clustering.evaluate_classification(
        mnist_data["trainY"].astype(np.int32),
        mnist_data["testX"].astype(np.int32),
        mnist_data["testY"].astype(np.int32),
    )
    result2["mnist"]["clustering_nmi"].append(mnist_nmi)
    result2["mnist"]["classification_accuracy"].append(mnist_acc)

# Task 3 - Semi-supervised Learning

# Overall Results

In [ ]:
plot_result(result1=result1, result2=result2)